In [1]:
#https://www.kaggle.com/code/italomarcelo/nlp-spacy-portuguese

! pip install -U spacy
! python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-21 16:05:16.929005: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
import pandas as pd

import re
import string 
import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
nltk.download('punkt')
punctuation = set(string.punctuation)

#lemma in portuguese
import spacy
nlp = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
#get binary_data

binary_data = pd.read_csv('https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_binary_classification.csv')
print(binary_data.shape)
binary_data.head(2)

(5670, 8)


,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C


In [5]:
#get hierarchical_data

hierarchical_data = pd.read_csv('https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv')
print(hierarchical_data.shape)
print(hierarchical_data.shape)
hierarchical_data.head(2)

(5668, 80)
(5668, 80)


,text,Hate.speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Thin.women,Arabic,East.europeans,Africans,South.Americans,Brazilians,Migrants,Homossexuals,Thin.people,Ageing
0,"""não come mel, morde marimbondo""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"não tem pinto, tem orgulho !",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#get hierarchical_classes

hierarchical_classes = pd.read_csv('https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/graph_hierarchical_classes.csv')
print(hierarchical_classes.shape)
print(hierarchical_data.shape)
hierarchical_classes.head(2)

(99, 2)
(5668, 80)


,Source,Target
0,Hate speech,Racism
1,Hate speech,Origin


In [7]:
#create hash function

import hashlib
def hash_string(text):
    ''''Transform string into a hash'''
    return {hashlib.sha1(text.strip().lower().encode("utf-8")).hexdigest(): text}

# hash_string("bb")

In [8]:
#extract text

binary = binary_data.text
hierarchical = hierarchical_data.text

In [9]:
#hashing hierarchical data

d = {}
for i in hierarchical:
    d.update(hash_string(i))

In [10]:
#checking what from binary is not in hierarchical 

l_result = []

for i in binary:
    #hashing binary data
    result = hash_string(i)
    if list(result.keys())[0] not in d:
        l_result.append(result)

In [11]:
#number of tweets in binary that are not in hierarchical

len(l_result)

666

In [12]:
# pre-processing

stopwords_manual = ['http?', 'mais', 'is?o', 'es[st]?*', 'quan[dt]?', ' ', '\n', '...', 'de o', 'em o', 'rt', 'ter', 'pra', 'a o', 'q', '  ', '..', 'por 0', 'fazer', 'dizer', 'vc']

def preprocessing(text):

    text = re.sub('@[\w]+','',text) #remove usernames
    text = re.sub(r'http\S+', '', text) #remove links
    text = [w.lemma_ for w in nlp(text)] #This lemma also performs tokenization?
    text = [word for word in text if word not in punctuation]
    text = [w.lower() for w in text]
    text = [word for word in text if word not in stopwords and word not in stopwords_manual]

    return text

In [13]:
#note: can only pre-process data once since it changes the type
hierarchical_data['text'] = hierarchical_data.text.apply(preprocessing)

In [14]:
bagOfWords = []
for tweet in hierarchical_data['text']:
    for word in tweet:
        bagOfWords.append(word)

numTokens = len(bagOfWords)
vocabSize = len(set(bagOfWords))

#tyep to token ratio
print("corpus size: ", numTokens)
print('vocabulary size: ', vocabSize)
print(" vocab to Corpus: ", vocabSize/numTokens)

top25 = nltk.FreqDist(bagOfWords).most_common(25)

print(top25)

corpus size:  47921
vocabulary size:  11104
 vocab to Corpus:  0.23171469710565307
[('mulher', 646), ('ir', 552), ('dia', 392), ('querer', 284), ('ver', 251), ('poder', 249), ('falar', 246), ('todo', 246), ('branco', 243), ('bom', 213), ('por o', 209), ('achar', 205), ('saber', 199), ('dar', 193), ('homem', 183), ('refugiado', 163), ('agora', 162), ('aqui', 161), ('pq', 160), ('feio', 158), ('gente', 145), ('gordar', 145), ('burra', 139), ('nao', 130), ('contra', 129)]


In [16]:
wordfreq = {}
for tweet in hierarchical_data['text']:
    for word in tweet:
        if word not in wordfreq.keys():
            wordfreq[word] = 1
        else:
            wordfreq[word] += 1

import operator
wordfreq = dict(sorted(wordfreq.items(), key=operator.itemgetter(1),reverse=True))
print (wordfreq)

{'mulher': 646, 'ir': 552, 'dia': 392, 'querer': 284, 'ver': 251, 'poder': 249, 'falar': 246, 'todo': 246, 'branco': 243, 'bom': 213, 'por o': 209, 'achar': 205, 'saber': 199, 'dar': 193, 'homem': 183, 'refugiado': 163, 'agora': 162, 'aqui': 161, 'pq': 160, 'feio': 158, 'gente': 145, 'gordar': 145, 'burra': 139, 'nao': 130, 'contra': 129, 'país': 129, 'ficar': 127, 'pnr': 127, 'sobre': 126, 'cara': 125, 'tá': 122, 'nada': 121, 'ano': 119, 'bem': 117, 'sapatão': 117, 'ainda': 116, 'hoje': 115, 'esquerda': 115, 'pessoa': 113, 'grande': 113, 'angola': 107, 'outro': 104, 'porque': 103, 'passar': 103, 'feliz': 103, 'chamar': 102, 'trump': 102, 'coisa': 101, 'dever': 101, 'portugal': 101, 'mundo': 99, 'tudo': 97, 'nunca': 92, 'sim': 92, 'n': 92, 'feminista': 92, 'brasil': 90, 'ta': 85, 'verdade': 84, 'assim': 84, 'usar': 83, 'negro': 83, 'bolsonaro': 83, 'novo': 82, 'vida': 81, '....': 76, 'sempre': 76, 'casa': 76, 'islão': 76, '\n ': 75, 'vez': 75, 'fufa': 75, 'gostar': 74, 'tão': 74, 'para